In [1]:
from docscraping import DocScraper
import requests
import re
import html
from cikscraping import CIKScraper
from webscraping import SECScraper
from sec_api import ExtractorApi
import pandas as pd

In [34]:
headers = {"User-Agent": "bxie43@wisc.edu"}
FILE = '10-K'
Year = '2023'
api_key = 'eea8f0562b43a7376c4374da9356914f6abf810c296f520d183c6b2de2708be7'

In [5]:
master_idx = pd.read_csv("Master_sheet - Sheet1.csv")
company_ticker = master_idx['company_ticker']

In [38]:
regulatory_compliance = pd.DataFrame(columns=['ticker', 'year', 'file', 'regulatory_compliance'])
product_portfolio = pd.DataFrame(columns=['ticker', 'year', 'file', 'product_portfolio'])
acquisition_info = pd.DataFrame(columns=['ticker', 'year', 'file', 'acquisition_info'])

for ticker in company_ticker:
    try:
        company = CIKScraper(ticker, headers)
        company.parsing_tickers()
        scraper = SECScraper(company.cik, Year, FILE, headers)
        url = scraper.scrape_sec_data()[0]

        try:
            parser = DocScraper(url, FILE, headers, api_key, Year, ticker)
            parser.parsing_file()


            # regulatory_compliance
            data_rc = {
            'ticker': ticker,
            'year': Year,
            'file': FILE,
            'regulatory_compliance': [parser.regulatorycompliance]
            }

            df_rc = pd.DataFrame(data_rc)
            regulatory_compliance = pd.concat([regulatory_compliance, df_rc], axis = 0)


            # product_portfolio
            data_pp = {
            'ticker': ticker,
            'year': Year,
            'file': FILE,
            'product_portfolio': [parser.productportfolio]
            }
            df_pp = pd.DataFrame(data_pp)
            product_portfolio = pd.concat([product_portfolio, df_pp], axis = 0)

            # acquisitions information
            data_ai = {
            'ticker': ticker,
            'year': Year,
            'file': FILE,
            'acquisition_info': [parser.acquisitions]
            }
            df_ai = pd.DataFrame(data_ai)
            acquisition_info = pd.concat([acquisition_info, df_ai], axis = 0)

        except Exception as e:
            print(e)

    except Exception:
        pass


In [56]:
regulatory_compliance.to_excel('regulatory_compliance.xlsx', index = False)

In [41]:
acquisition_info.to_excel('acquisition_info.xlsx', index = False)

In [48]:
product_portfolio.to_excel('product_portfolio.xlsx', index=False)